In [29]:
import time

In [30]:
import json

In [31]:
from datetime import datetime

In [32]:
import nexussdk as nxs

In [33]:
# Example:
# https://nexus.domain.org/v1
with open("DEPLOYMENT", "r") as f:
    DEPLOYMENT = f.readline().rstrip()
    nxs.config.set_environment(DEPLOYMENT)

In [34]:
with open("TOKEN", "r") as f:
    TOKEN = f.readline().rstrip()
    nxs.config.set_token(TOKEN)

In [35]:
pp = nxs.tools.pretty_print

# Examples setup

In [36]:
def utcnow() -> str:
    return datetime.utcnow().strftime("%Y%m%d%H%M%S%f")

In [37]:
def generate_compacted_id() -> str:
    return "nxv:storage{}".format(utcnow())

In [38]:
def generate_expanded_id() -> str:
    return "{}/storage{}".format("https://knowledgegraph.domain.org", utcnow())

In [39]:
ORGANIZATION = "myorg{}".format(utcnow())

In [40]:
nxs.organizations.create(ORGANIZATION)["@id"]

'http://dev.nexus.ocp.bbp.epfl.ch/v1/orgs/myorg20190522150300907699'

In [41]:
PROJECT = "myproject"

In [42]:
nxs.projects.create(ORGANIZATION, PROJECT)["@id"]

'http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907699/myproject'

In [43]:
with open("S3_ENDPOINT") as f:
    S3_ENDPOINT = f.readline().rstrip()

In [44]:
with open("S3_ACCESS_KEY") as f:
    S3_ACCESS_KEY = f.readline().rstrip()

In [45]:
with open("S3_SECRET_KEY") as f:
    S3_SECRET_KEY = f.readline().rstrip()

In [ ]:
with open("EXTERNAL_DISK_ENDPOINT") as f:
    EXTERNAL_DISK_ENDPOINT = f.readline().rstrip()

In [ ]:
with open("EXTERNAL_DISK_CREDENTIALS") as f:
    EXTERNAL_DISK_CREDENTIALS = f.readline().rstrip()

### User API

In [46]:
s3_storage_id = nxs.storages.create_s3_storage(
    ORGANIZATION,
    PROJECT,
    "sdkbucket",
    endpoint = S3_ENDPOINT,
    access_key = S3_ACCESS_KEY,
    secret_key = S3_SECRET_KEY)["@id"]

In [ ]:
disk_storage_id = nxs.storages.create_disk_storage(ORGANIZATION, PROJECT, "/tmp")["@id"]

In [ ]:
external_disk_storage_id = nxs.storages.create_external_disk_storage(
    ORGANIZATION,
    PROJECT,
    EXTERNAL_DISK_ENDPOINT,
    "testproject",
    credentials=EXTERNAL_DISK_CREDENTIALS)["@id"]

In [19]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, s3_storage_id))

{
  "@context": [
    "https://bluebrain.github.io/nexus/contexts/storage.json",
    "https://bluebrain.github.io/nexus/contexts/resource.json"
  ],
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522145317832858/myproject/_/dc2e42de-30aa-42dc-a4de-689c19899cb5",
  "@type": [
    "S3Storage",
    "Storage"
  ],
  "bucket": "sdkbucket",
  "default": false,
  "endpoint": "http://minio.dev.nexus.ocp.bbp.epfl.ch",
  "readPermission": "resources/read",
  "writePermission": "files/write",
  "_algorithm": "SHA-256",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/storages/myorg20190522145317832858/myproject/dc2e42de-30aa-42dc-a4de-689c19899cb5",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/storage.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522145317832858/myproject",
  "_rev": 1,
  "_deprecated": false,
  "_createdAt": "2019-05-22T14:53:26.647Z",
  "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/github/users/henry

In [ ]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, disk_storage_id))

In [ ]:
pp(nxs.storages.fetch(ORGANIZATION, PROJECT, external_disk_storage_id))

In [20]:
pp(nxs.storages.list(ORGANIZATION, PROJECT))

{
  "@context": [
    "https://bluebrain.github.io/nexus/contexts/search.json",
    "https://bluebrain.github.io/nexus/contexts/resource.json"
  ],
  "_total": 2,
  "_results": [
    {
      "@id": "https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault",
      "@type": [
        "https://bluebrain.github.io/nexus/vocabulary/Storage",
        "https://bluebrain.github.io/nexus/vocabulary/DiskStorage"
      ],
      "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/storages/myorg20190522144800398445/myproject/https%3A%2F%2Fbluebrain.github.io%2Fnexus%2Fvocabulary%2FdiskStorageDefault",
      "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/storage.json",
      "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522144800398445/myproject",
      "_rev": 1,
      "_deprecated": false,
      "_createdAt": "2019-05-22T14:48:05.638Z",
      "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/github/users/henry",
      "_updatedAt": "2019-05-22T14:48:05.

In [ ]:
pp(nxs.storages.list(ORGANIZATION, PROJECT, type="nxv:S3Storage"))

In [ ]:
pp(nxs.storages.tag(ORGANIZATION, PROJECT, disk_storage_id, tag="v1.1.1", rev_to_tag=1, rev=1))

In [ ]:
pp(nxs.storages.tags(ORGANIZATION, PROJECT, disk_storage_id))

## Files API

In [47]:
linked_image = nxs.files.create_link(ORGANIZATION, PROJECT, "idontalways.jpg", "6emgw.jpg", "image/jpeg", storage_id=s3_storage_id)

In [21]:
pp(linked_image)

{
  "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522145317832858/myproject/_/d13380bb-65e8-4071-835e-671c840d83f5",
  "@type": "File",
  "_storageId": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522145317832858/myproject/_/dc2e42de-30aa-42dc-a4de-689c19899cb5",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522145317832858/myproject/d13380bb-65e8-4071-835e-671c840d83f5",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522145317832858/myproject",
  "_rev": 1,
  "_deprecated": false,
  "

In [48]:
pp(nxs.files.fetch(ORGANIZATION, PROJECT, linked_image["@id"], out_filepath = linked_image["_filename"]))

{
  "@context": [
    {
      "@base": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/",
      "@vocab": "http://dev.nexus.ocp.bbp.epfl.ch/v1/vocabs/myorg20190522150300907699/myproject/"
    },
    "https://bluebrain.github.io/nexus/contexts/resource.json"
  ],
  "@id": "a771b5c6-0558-468c-aa89-82f6e086ce48",
  "@type": "File",
  "_storageId": "8affbb22-5ffd-462b-a8ec-b03248dc88ca",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907

![I don't always](idontalways.jpg)

In [49]:
pp(nxs.files.create(ORGANIZATION, PROJECT, "idontalways.jpg"))

{
  "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/f252984e-a81d-4c95-9e50-d65b2451a25d",
  "@type": "File",
  "_storageId": "nxv:diskStorageDefault",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/f252984e-a81d-4c95-9e50-d65b2451a25d",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907699/myproject",
  "_rev": 1,
  "_deprecated": false,
  "_createdAt": "2019-05-22T15:03:40.296Z",
  "_createdBy": "http://dev.nexus.ocp.bbp.epfl.ch/v1/realms/github/users/henry",
  "_updatedAt": "2019-05-22T15:03:40.296Z",
  "_updat

In [50]:
pp(nxs.files.update_link(ORGANIZATION, PROJECT, "idontalways.jpg", "6emgw.jpg", "image/jpeg", 1, file_id=linked_image["@id"], storage_id=s3_storage_id))

{
  "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "@type": "File",
  "_storageId": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/8affbb22-5ffd-462b-a8ec-b03248dc88ca",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907699/myproject",
  "_rev": 2,
  "_deprecated": false,
  "

In [51]:
pp(nxs.files.list(ORGANIZATION, PROJECT))

{
  "@context": [
    "https://bluebrain.github.io/nexus/contexts/search.json",
    "https://bluebrain.github.io/nexus/contexts/resource.json"
  ],
  "_total": 2,
  "_results": [
    {
      "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/a771b5c6-0558-468c-aa89-82f6e086ce48",
      "@type": "https://bluebrain.github.io/nexus/vocabulary/File",
      "_storageId": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/8affbb22-5ffd-462b-a8ec-b03248dc88ca",
      "_bytes": 28173,
      "_digest": {
        "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
        "_algorithm": "SHA-256"
      },
      "_filename": "idontalways.jpg",
      "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
      "_mediaType": "image/jpeg",
      "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
      "_con

In [52]:
pp(nxs.files.tag(ORGANIZATION, PROJECT, linked_image["@id"], 1, "original", rev=2))

{
  "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "@type": "File",
  "_storageId": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/8affbb22-5ffd-462b-a8ec-b03248dc88ca",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907699/myproject",
  "_rev": 3,
  "_deprecated": false,
  "

In [59]:
pp(nxs.files.fetch(ORGANIZATION, PROJECT, linked_image["@id"], tag="original"))

{
  "@context": [
    {
      "@base": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/",
      "@vocab": "http://dev.nexus.ocp.bbp.epfl.ch/v1/vocabs/myorg20190522150300907699/myproject/"
    },
    "https://bluebrain.github.io/nexus/contexts/resource.json"
  ],
  "@id": "a771b5c6-0558-468c-aa89-82f6e086ce48",
  "@type": "File",
  "_storageId": "8affbb22-5ffd-462b-a8ec-b03248dc88ca",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907

In [60]:
pp(nxs.files.deprecate(ORGANIZATION, PROJECT, linked_image["@id"], 3))

{
  "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
  "@id": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "@type": "File",
  "_storageId": "http://dev.nexus.ocp.bbp.epfl.ch/v1/resources/myorg20190522150300907699/myproject/_/8affbb22-5ffd-462b-a8ec-b03248dc88ca",
  "_bytes": 28173,
  "_digest": {
    "_value": "7a132bb63cb25f6f86d76ddf1c7ea3db785b1ccec6ca34d209c18dc4e08479a4",
    "_algorithm": "SHA-256"
  },
  "_filename": "idontalways.jpg",
  "_location": "http://minio.dev.nexus.ocp.bbp.epfl.ch/sdkbucket/6emgw.jpg",
  "_mediaType": "image/jpeg",
  "_self": "http://dev.nexus.ocp.bbp.epfl.ch/v1/files/myorg20190522150300907699/myproject/a771b5c6-0558-468c-aa89-82f6e086ce48",
  "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/file.json",
  "_project": "http://dev.nexus.ocp.bbp.epfl.ch/v1/projects/myorg20190522150300907699/myproject",
  "_rev": 4,
  "_deprecated": true,
  "_